# Mapping largest cities
## Experiments with folium, zipfile and pandas

In [13]:
import zipfile, requests, codecs
from io import BytesIO
import pandas as pd
import folium
from folium.plugins import MarkerCluster

%matplotlib inline

## Get online data from Census gazeteer

https://www.census.gov/geo/maps-data/data/gazetteer2010.html

In [2]:
url = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/Gaz_places_national.zip'
r = requests.get(url)
z = zipfile.ZipFile(BytesIO(r.content))
z.extractall()

There is just one file in this zipfile

In [3]:
filename = z.namelist()[0]
filename

'Gaz_places_national.txt'

I was getting an encoding error trying to read this file into a pandas dataframe. The following makes sure the file is encoded in utf-8.

In [4]:
file = codecs.open(filename, "r",encoding='utf-8', errors='ignore')
df = pd.read_table(file, sep = '\t')

Some column names might have whitespace in them, so strip those out.

In [5]:
df = df.rename(columns=lambda x: x.strip())

In [6]:
df.columns

Index(['USPS', 'GEOID', 'ANSICODE', 'NAME', 'LSAD', 'FUNCSTAT', 'POP10',
       'HU10', 'ALAND', 'AWATER', 'ALAND_SQMI', 'AWATER_SQMI', 'INTPTLAT',
       'INTPTLONG'],
      dtype='object')

In [7]:
df.head()

,USPS,GEOID,ANSICODE,NAME,LSAD,FUNCSTAT,POP10,HU10,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,100100,2582661,Abanda CDP,57,S,192,79,7764035,34284,2.998,0.013,33.091627,-85.527029
1,AL,100124,2403054,Abbeville city,25,A,2688,1255,40255352,107642,15.543,0.042,31.564216,-85.259634
2,AL,100460,2403063,Adamsville city,25,A,4522,1990,65083153,29719,25.129,0.011,33.605750,-86.974650
3,AL,100484,2405123,Addison town,43,A,758,351,9753286,83426,3.766,0.032,34.202689,-87.177901
4,AL,100676,2405125,Akron town,43,A,356,205,1778126,13850,0.687,0.005,32.879417,-87.741757


In [8]:
print('There are',len(df),'cities and towns in this database.')

There are 29514 cities and towns in this database.


Let's get just the cities with > 250K population.  Then we'll display a map of the USA with a marker for each of those cities using Folium.

We're automatically clustering the markers.  If we didn't want to do that we'd not use `marker_cluster` and instead `.ad_to(map)` each Marker.

In [9]:
big250 = df[['NAME', 'POP10', 'INTPTLAT', 'INTPTLONG']][df.POP10>250000].sort_values('POP10', ascending=False)

In [10]:
map = folium.Map(location=[30,-97], zoom_start=4,tiles='Stamen Terrain')

In [14]:
marker_cluster = MarkerCluster().add_to(map)

In [15]:
for city in big250.iterrows():
    folium.Marker(
        location = [city[1]['INTPTLAT'],city[1]['INTPTLONG']], 
        popup = city[1]['NAME']+ ' pop:'+str(city[1]['POP10'])
    ).add_to(marker_cluster)
                    
map